In [ ]:
import os
from dataclasses import fields
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from airo_camera_toolkit.point_clouds.conversions import point_cloud_to_open3d
from airo_dataset_tools.data_parsers.pose import Pose
from cloth_tools.annotation.grasp_annotation import GraspAnnotation
from cloth_tools.dataset.format import load_competition_observation
from cloth_tools.dataset.download import download_and_extract_dataset
from cloth_tools.visualization.opencv import draw_pose


data_dir = Path("../data")
dataset_dir = data_dir / "cloth_competition_dataset_0001_uncleaned"

In [ ]:
os.path.exists(dataset_dir)

In [ ]:
def emoji(dir: str, file: str) -> str:
    if os.path.isdir(os.path.join(dir, file)):
        return "📁"
    elif file.endswith(".jpg") or file.endswith(".png"):
        return "🖼️"
    elif file.endswith(".mp4"):
        return "🎥"
    return "📄"

print("First directories in the dataset:")
for f in sorted(os.listdir(dataset_dir))[:5]:
    print(emoji(dataset_dir, f), f)

In [ ]:
sample_dir = dataset_dir / "sample_000000"

for f in os.listdir(sample_dir):
    print(emoji(sample_dir, f), f)

In [ ]:
count_with_observation_start = 0
count_with_observation_end = 0
count_with_grasp = 0
count_with_all = 0

for sample_dir in os.listdir(dataset_dir):
    if not os.path.isdir(dataset_dir / sample_dir):
        continue
    if not sample_dir.startswith("sample_"):
        continue

    sample_dir = dataset_dir / sample_dir
    grasp_dir = sample_dir / "grasp"
    observation_start_dir = sample_dir / "observation_start"
    observation_result_dir = sample_dir / "observation_result"
    if os.path.exists(grasp_dir):
        count_with_grasp += 1

    if os.path.exists(observation_start_dir):
        count_with_observation_start += 1

    if os.path.exists(observation_result_dir):
        count_with_observation_end += 1

    if os.path.exists(grasp_dir) and os.path.exists(observation_start_dir) and os.path.exists(observation_result_dir):
        count_with_all += 1

print(f"Total episodes started: {len(os.listdir(dataset_dir))}")
print(f"Count with observation_start: {count_with_observation_start}")
print(f"Count with grasp: {count_with_grasp}")
print(f"Count with observation_end: {count_with_observation_end}")
print(f"Count with all: {count_with_all}")
print(f"Grasp execution success rate: {count_with_observation_end / count_with_grasp:.2f}")
print(f"Grasp success rate: {count_with_observation_end / count_with_observation_start:.2f}")

In [ ]:
dataset_cleaned_dir = data_dir / "cloth_competition_dataset_0001_cleaned"

In [ ]:
import shutil
from loguru import logger

i_cleaned = 0
os.makedirs(dataset_cleaned_dir, exist_ok=True)


for sample_dir in sorted(os.listdir(dataset_dir)):
    if not os.path.isdir(dataset_dir / sample_dir):
        continue
    if not sample_dir.startswith("sample_"):
        continue

    sample_dir = dataset_dir / sample_dir
    grasp_dir = sample_dir / "grasp"
    observation_start_dir = sample_dir / "observation_start"
    observation_result_dir = sample_dir / "observation_result"

    if not (
        os.path.exists(grasp_dir) and os.path.exists(observation_start_dir) and os.path.exists(observation_result_dir)
    ):
        continue

    logger.info(f"Copying {sample_dir}")

    sample_cleaned_dir = dataset_cleaned_dir / f"sample_{i_cleaned:06d}"
    # all dirs exist -> copy to cleaned

    src = sample_dir
    dst = sample_cleaned_dir
    shutil.copytree(src, dst)

    i_cleaned += 1

In [ ]:
# attempt to load all data in the cleaned directory to identify any issues

for sample_dir in sorted(os.listdir(dataset_cleaned_dir)):
    sample_dir = dataset_cleaned_dir / sample_dir
    grasp_dir = sample_dir / "grasp"
    observation_start_dir = sample_dir / "observation_start"
    observation_result_dir = sample_dir / "observation_result"

    logger.info(f"Loading {sample_dir}")
    observation_start = load_competition_observation(observation_start_dir)
    observation_result = load_competition_observation(observation_result_dir)

    image_frontal_annotated = cv2.imread(str(grasp_dir / "frontal_image_grasp.jpg"))
    image_top_annotated = cv2.imread(str(grasp_dir / "topdown_image_grasp.jpg"))

    grasp_pose_file = grasp_dir / "grasp_pose.json"
    grasp_annotation_file = grasp_dir / "grasp_annotation.json"

    with open(grasp_pose_file, "r") as f:
        grasp_pose = Pose.model_validate_json(f.read()).as_homogeneous_matrix()


    with open(grasp_annotation_file, "r") as f:
        grasp_annotation = GraspAnnotation.model_validate_json(f.read())


    